## APILayer

Чтобы перевести платежи из валюты страны в доллары США используется функция currency_change, которая полачает данные по курсу валют из APILayer

In [1]:
import requests
import pandas as pd
import datetime as dt
import json
import os
from dotenv import load_dotenv,find_dotenv

In [2]:
load_dotenv(dotenv_path='Overmobile/.env')

API_KEY_LAYER = os.getenv('API_KEY_LAYER')

In [3]:
def currency_change(start_date: dt.datetime, end_date: dt.datetime, payments: pd.DataFrame):
    """
    Функция принимает на вход:
    start_date - дата начала выгрузки
    end_date - дата конца выгрузки 
    payments - платежи в разных валютах (ВЕБ ПЛАТЕЖИ ДОЛЖНЫ БЫТЬ ПЕРЕВЕДНЫ В РУБЛИ ЗАРАНЕЕ) 

    На выходе получаем 3 dataframe: 
    layer_currency -  курсы валют для каждого дня из отрезка [start_date, end_date], 
    payments - платежи пользователей в долларах США для каждого дня из отрезка [start_date, end_date], 
    """

    charcode_array = ','.join(
        [str(i) for i in payments.charcode.unique()])  # массив валют

    # дата в нужном для запросе формате
    url_start = start_date.strftime("%Y-%m-%d")
    url_end = end_date.strftime("%Y-%m-%d")

    # воспользуемся apilayer для выгрузки котировок валют за нужный период времени
    url = f"https://api.apilayer.com/fixer/timeseries?start_date={url_start}&end_date={url_end}&base=USD&symbols={charcode_array}"

    payload = {}
    headers = {"apikey": API_KEY_LAYER}
    response = requests.request("GET", url, headers=headers, data=payload)

    status_code = response.status_code
    result = response.text
    data = json.loads(result)  # прочитаем как json - получим словарь

    # выберем по ключу из словаря нужные данные и запишем в df
    layer_currency = pd.DataFrame(data['rates'])
    layer_currency = layer_currency.stack()  # сделаем пары: валюта, день

    layer_currency = pd.DataFrame(layer_currency)

    # сделаем из идекса столбец два раза
    layer_currency.reset_index(level=0, inplace=True)
    layer_currency.reset_index(level=0, inplace=True)

    # переименут столбцы 
    layer_currency = layer_currency.rename(
        columns={'index': 'date', 'level_0': 'charcode', 0: 'value'})

    layer_currency['date'] = pd.to_datetime(
        layer_currency['date'], format="%Y-%m-%d")

    return layer_currency
